<a href="https://colab.research.google.com/github/UN-GCPDS/Curso-Corto-LLMs/blob/main/3.%20Dashboard/Generaci%C3%B3n_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Logo UNAL CHEC](https://github.com/UN-GCPDS/curso_IA_CHEC/blob/main/logo_unal_chec.jpg?raw=1)

# **Generación de datos**

## **Descripción**

Entrenamiento de modelo Tabnet bajo diversas condiciones.

### **Profesor - Sesión 1:** Andrés Marino Álvarez Meza y Diego Armando Pérez Rosero

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Datos

**TabNet para criticidad en redes de media tensión — Planteamiento y datos (Regresión)**

Sea el conjunto de datos

$$
\mathbf{X}\in\mathbb{R}^{N\times M},\qquad
\mathbf{y}\in\mathbb{R}^{N}.
$$

Cada fila de $\mathbf{X}$ representa un evento o periodo entre 2019 y 2024 y contiene las características de los elementos asociados al equipo que operó. El vector $\mathbf{y}$ almacena el valor continuo del indicador a modelar (SAIDI o SAIFI) para ese mismo evento/periodo.

Definimos

$$
\mathcal{F}:\mathcal{X}\subseteq\mathbb{R}^{M}\to\mathbb{R},\qquad
\hat{y}=\mathcal{F}(\mathbf{x})
=
\bigl(\,\breve{f}_{L}\circ \breve{f}_{L-1}\circ \cdots \circ \breve{f}_{1}\,\bigr)(\mathbf{x}),
$$

donde $\breve{f}_{l}(\cdot)$ denota el $l$-ésimo bloque del modelo ($l\in\{1,\dots,L\}$) y $\circ$ es el operador de composición.

En caso multisalida para $(\text{SAIDI},\text{SAIFI})$, se toma $\mathcal{F}:\mathbb{R}^{M}\to\mathbb{R}^{2}$ y $\mathbf{y}\in\mathbb{R}^{N\times 2}$.
![Logo UNAL CHEC](https://raw.githubusercontent.com/Daprosero/Deep-Convolutional-Generative-Adversarial-Network/refs/heads/master/Mercados%20CHEC.png)

In [ ]:
#@title Librerías
# Instalación de paquetes necesarios
!pip install -q gdown
!pip install openTSNE
!pip install pytorch-tabnet optuna
!pip install wget --quiet

# Importación de librerías necesarias
import optuna
import warnings
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, QuantileTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.neighbors import NearestNeighbors
from pytorch_tabnet.tab_model import TabNetRegressor, TabNetClassifier
from pytorch_tabnet.augmentations import RegressionSMOTE
from google.colab import drive
import tensorflow as tf
import tensorflow_probability as tfp
import os
from pathlib import Path
import math
import wget
!gdown --id 1o_fZIhk6ErrtrM3eVZPF9s2qj8l4FoqS -O SuperEventos_Criticidad_AguasAbajo_CODEs.zip
!gdown --id 1lBrseLoEmr6-VwNSCHOp2zuc4sKKrkbQ -O model.zip
!gdown --id 16VIuHLgPGpX4J723Wd48UAPhHivLuUaH -O Data_CHEC.zip

import zipfile
import os

zip_path = "SuperEventos_Criticidad_AguasAbajo_CODEs.zip"
extract_dir = "CHEC"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

zip_path = "model.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

zip_path = "Data_CHEC.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Supresión de warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="pandas")
warnings.filterwarnings("ignore", category=FutureWarning)

# Función auxiliar para etiquetas
def get_labels(x: pd.Series) -> pd.Series:
    labels, _ = pd.factorize(x)
    return pd.Series(labels, name=x.name, index=x.index)

# Definición de funciones personalizadas de pérdida
def my_mse_loss_fn(y_pred, y_true):
    mse_loss = (y_true - y_pred) ** 2
    return torch.mean(mse_loss)
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def plot_var_band(
    df,
    var_token,
    row_index=0,
    hours_back=24,
    col_patterns=None,
    display_name=None,
    units=None,
    event_label="evento reportado",
):
    """
    Grafica una variable climática en una franja de horas hacia atrás.

    Parámetros
    ----------
    df : pd.DataFrame
        Contiene columnas por hora para la variable elegida.
        Ejemplos de nombres soportados automáticamente:
        - 'h0-<var>', 'h1-<var>', ..., 'h24-<var>'
        - '<var>_h0', '<var>_h1', ...
        con separadores '_' o '-'.

    var_token : str
        Nombre base de la variable en los nombres de columna (p.ej. 'wind_gust_spd',
        'air_temp', 'precip'). Debe coincidir con lo que aparece en las columnas.

    row_index : int
        Fila (evento) a graficar.

    hours_back : int
        Cuántas horas hacia atrás mostrar.

    col_patterns : list[str] | None
        Lista de regex opcionales para detectar columnas por hora.
        Si None, se generan automáticamente a partir de var_token.

    display_name : str | None
        Etiqueta legible para el eje Y (p.ej. 'Ráfaga de viento').
        Si None, se usa var_token.

    units : str | None
        Unidades para concatenar en la etiqueta Y (p.ej. 'm/s', '°C', 'mm').

    event_label : str
        Texto para la flecha en la hora 0.
    """
    # --- 1) Preparar patrones de columnas ---
    if col_patterns is None:
        # Permitir '_' o '-' (o espacio) entre partes del var_token
        parts = re.split(r'[_\-\s]+', var_token.strip())
        # Construimos un regex que tolere '_' o '-' entre partes
        # ej: 'wind[_-]?gust[_-]?spd'
        var_regex = r'[_-]?'.join(map(re.escape, parts))

        col_patterns = [
            rf'^h(\d{{1,2}})[-_]?{var_regex}$',   # h0-<var>  o  h0_<var>
            rf'^{var_regex}[-_]?h(\d{{1,2}})$',   # <var>-h0  o  <var>_h0
        ]

    # --- 2) Detectar columnas y mapear a hora ---
    hour_to_col = {}
    for c in df.columns:
        for pat in col_patterns:
            m = re.match(pat, str(c), flags=re.IGNORECASE)
            if m:
                h = int(m.group(1))
                hour_to_col[h] = c
                break

    if not hour_to_col:
        raise ValueError(
            f"No se encontraron columnas con horas para la variable '{var_token}'.\n"
            f"Prueba ajustando 'var_token' o pasando 'col_patterns' personalizados."
        )

    # --- 3) Construir serie horas [0..hours_back] si existen, orden ascendente ---
    hours = [h for h in sorted(hour_to_col.keys()) if 0 <= h <= hours_back]
    vals = np.array(
        [pd.to_numeric(df.loc[df.index[row_index], hour_to_col[h]], errors='coerce') for h in hours],
        dtype=float
    )

    # --- 4) Graficar ---
    fig, ax = plt.subplots(figsize=(10, 5))

    # línea y puntos
    ax.plot(hours, vals, marker='o')

    # invertir eje X para que se vea 24 -> 0
    ax.set_xlim(hours_back, 0)

    # franja sombreada
    ymin = np.nanmin(vals) if np.isfinite(np.nanmin(vals)) else 0.0
    ymax = np.nanmax(vals) if np.isfinite(np.nanmax(vals)) else 1.0
    pad  = 0.05 * (ymax - ymin if ymax > ymin else 1.0)
    ax.set_ylim(ymin - pad, ymax + pad)
    ax.axvspan(0, hours_back, alpha=0.15)

    # flecha y etiqueta en hora 0
    y0 = vals[hours.index(0)] if 0 in hours else np.nan
    if not np.isfinite(y0):
        y0 = np.nanmedian(vals) if np.isfinite(np.nanmedian(vals)) else (ymin + ymax) / 2.0

    ax.annotate(
        event_label,
        xy=(0, y0),
        xytext=(max(2, min(4, hours_back*0.15)), y0 + (ymax - y0)*0.15),
        arrowprops=dict(arrowstyle="->", lw=1),
        ha='left', va='bottom'
    )

    # etiquetas
    ylab = display_name if display_name else var_token
    if units:
        ylab = f"{ylab} [{units}]"
    ax.set_xlabel("Horas antes del evento")
    ax.set_ylabel(ylab)
    ax.grid(True, alpha=0.3)

    # ticks principales (24, 18, 12, 6, 0) si corresponde
    xticks = [h for h in [hours_back, 18, 12, 6, 0] if 0 <= h <= hours_back]
    ax.set_xticks(xticks)

    plt.tight_layout()
    plt.show()


# --- Ejemplo de uso:
# plot_wind_gust_band(df=tu_dataframe, row_index=0, hours_back=24)

def my_rmse_loss_fn(y_pred, y_true):
    mse_loss = (y_true - y_pred) ** 2
    mean_mse_loss = torch.mean(mse_loss)
    rmse_loss = torch.sqrt(mean_mse_loss)
    return rmse_loss

def my_mae_loss_fn(y_pred, y_true):
    mae_loss = torch.abs(y_true - y_pred)
    return torch.mean(mae_loss)

def my_mape_loss_fn(y_pred, y_true):
    mape_loss = torch.abs((y_true - y_pred) / y_true) * 100
    return torch.mean(mape_loss)

def my_r2_score_fn(y_pred, y_true):
    total_variance = torch.var(y_true, unbiased=False)
    unexplained_variance = torch.mean((y_true - y_pred) ** 2)
    r2_score = 1 - (unexplained_variance / total_variance)
    return 1-r2_score

# Etapa 0: imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
# ==== Librerías ====
import numpy as np
import cupy as cp
import xgboost as xgb

from cuml.ensemble import RandomForestRegressor as cuRF
from cuml.metrics import r2_score as r2_gpu

# Si quieres comparar con CPU para sanity-check:
from sklearn.metrics import r2_score as r2_cpu
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
# ==== Utilidades ====
def to_cpu(a):
    """Convierte CuPy -> NumPy si aplica."""
    try:
        if isinstance(a, cp.ndarray):
            return cp.asnumpy(a)
    except Exception:
        pass
    return a

def metrics_gpu(y_true_cp, y_pred_cp):
    """MAE, RMSE, R2 calculados en GPU (CuPy)."""
    y_true_cp = cp.asarray(y_true_cp)
    y_pred_cp = cp.asarray(y_pred_cp)
    mae  = float(cp.mean(cp.abs(y_true_cp - y_pred_cp)))
    rmse = float(cp.sqrt(cp.mean((y_true_cp - y_pred_cp)**2)))
    ssr  = float(cp.sum((y_true_cp - y_pred_cp)**2))
    sst  = float(cp.sum((y_true_cp - cp.mean(y_true_cp))**2))
    r2   = 1.0 - ssr / sst if sst > 0 else np.nan
    return mae, rmse, r2

def permutation_importance_rf_gpu(model, X_val_cp, y_val_cp, n_repeats=3, max_feats=None, random_state=42):
    """
    Permutation importance en GPU para RF cuML.
    Devuelve importancia por feature (drop medio de R2 en valid).
    Si max_feats no es None, calcula solo para las primeras max_feats columnas (para acelerar).
    """
    rs = cp.random.RandomState(random_state)
    X_val_cp = cp.asarray(X_val_cp)
    y_val_cp = cp.asarray(y_val_cp)

    # R2 base
    y_pred_base = model.predict(X_val_cp)
    _, _, r2_base = metrics_gpu(y_val_cp, y_pred_base)

    n, d = X_val_cp.shape
    d_eval = d if max_feats is None else int(min(max_feats, d))
    importances = cp.zeros(d, dtype=cp.float32)

    for j in range(d_eval):
        drops = []
        for _ in range(n_repeats):
            Xp = X_val_cp.copy()
            idx = rs.permutation(n)
            Xp[:, j] = Xp[idx, j]  # permutar solo la columna j
            y_pred_p = model.predict(Xp)
            _, _, r2_p = metrics_gpu(y_val_cp, y_pred_p)
            drops.append(r2_base - r2_p)
        importances[j] = cp.mean(cp.asarray(drops))

    return importances  # CuPy array
def regression_metrics(y_true, y_pred):
    mae  = float(np.mean(np.abs(y_true - y_pred)))
    rmse = float(np.sqrt(np.mean((y_true - y_pred)**2)))
    ss_res = float(np.sum((y_true - y_pred)**2))
    ss_tot = float(np.sum((y_true - np.mean(y_true))**2))
    r2 = 1 - ss_res/ss_tot if ss_tot > 0 else np.nan
    return mae, rmse, r2
class CustomTabNetRegressor(TabNetRegressor):
    def __init__(self, *args, **kwargs):
        super(CustomTabNetRegressor, self).__init__(*args, **kwargs)

    def forward(self, X):
        output, M_loss = self.network(X)
        output = torch.relu(output)
        return output, M_loss

    def predict(self, X):
        device = next(self.network.parameters()).device
        if not isinstance(X, torch.Tensor):
            X = torch.tensor(X, dtype=torch.float32)
        X = X.to(device)
        with torch.no_grad():
            output, _ = self.forward(X)
        return output.cpu().numpy()
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree
from tqdm import tqdm
from ast import literal_eval
from pandas.api.types import is_numeric_dtype


import numpy as np
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
def make_strat_labels(y_vals, n_bins=3, min_per_class=2):
    """
    Genera etiquetas para estratificar a partir de un objetivo continuo.
    Reduce bins si no hay suficientes muestras por clase.
    """
    y1d = y_vals.reshape(-1)
    for bins in range(n_bins, 1, -1):
        pct = np.linspace(0, 100, bins + 1)[1:-1]
        cuts = np.percentile(y1d, pct)
        if np.any(np.diff(cuts) <= 0):
            continue
        labels = np.digitize(y1d, bins=cuts).astype(int)
        counts = Counter(labels)
        if all(c >= min_per_class for c in counts.values()) and len(counts) > 1:
            return labels
    return None

def stratify_from_df_or_y(df_labels, idx, y_subset, col='NIVEL_C'):
    """Intenta usar df[col] como etiqueta; si falla, usa percentiles en y_subset."""
    try:
        ycat_full = df_labels.loc[:, col].values.astype(int)
        ycat = ycat_full[idx]
        c10 = Counter(ycat)
        if all(v >= 2 for v in c10.values()) and len(c10) > 1:
            return ycat
    except Exception:
        pass
    return make_strat_labels(y_subset[:,0], n_bins=3, min_per_class=2)

def split_subset(X, y, df_labels=None, n_sub=1000, test_size=0.20, seed=42):
    """
    1) Toma un subset aleatorio de tamaño n_sub.
    2) Escala y (MinMax) sobre el subset.
    3) Split train/test con estratificación si es viable.
    4) Split train/valid (20% del train), con re-estratificación si es posible.
    """
    rng = np.random.RandomState(seed)
    n_total = X.shape[0]
    n_sub = min(n_sub, n_total)
    idx_sub = rng.choice(n_total, size=n_sub, replace=False)

    X_sub = X[idx_sub]
    y_sub = y[idx_sub]
    # etiquetas auxiliares para estratificación
    ycat_sub = stratify_from_df_or_y(df_labels, idx_sub, y_sub) if df_labels is not None else make_strat_labels(y_sub[:,0])
    # escalar objetivo en el subset
    scaler = MinMaxScaler()
    y_sub_scaled = scaler.fit_transform(y_sub)

    split_kwargs = dict(test_size=test_size, random_state=seed, shuffle=True)
    if ycat_sub is not None:
        X_tr, X_te, y_tr, y_te, ycat_tr, ycat_te = train_test_split(
            X_sub, y_sub_scaled, ycat_sub, stratify=ycat_sub, **split_kwargs
        )
    else:
        X_tr, X_te, y_tr, y_te = train_test_split(X_sub, y_sub_scaled, **split_kwargs)
        ycat_tr = ycat_te = None

    # Validación (20% del train)
    if ycat_tr is not None:
        y_tr_raw = y_tr[:,0]
        ycat_t = make_strat_labels(y_tr_raw, n_bins=3, min_per_class=2)
        if ycat_t is not None:
            X_tr, X_va, y_tr, y_va, ycat_tr, ycat_va = train_test_split(
                X_tr, y_tr, ycat_tr, test_size=0.20, random_state=seed, stratify=ycat_t
            )
        else:
            X_tr, X_va, y_tr, y_va = train_test_split(
                X_tr, y_tr, test_size=0.20, random_state=seed, shuffle=True
            )
            ycat_va = None
    else:
        X_tr, X_va, y_tr, y_va = train_test_split(
            X_tr, y_tr, test_size=0.20, random_state=seed, shuffle=True
        )
        ycat_va = None

    # Reporte rápido
    print("Originales (conservados):", X_orig.shape, y_orig.shape)
    print(f"Subset de {n_sub}:", X_sub.shape, y_sub.shape)
    print("Train/Valid/Test:", X_tr.shape, X_va.shape, X_te.shape)
    if ycat_sub is not None:
        print("Distribución clases subset:", Counter(ycat_sub))

    return {
        "idx_sub": idx_sub,
        "X_train": X_tr, "X_valid": X_va, "X_test": X_te,
        "y_train": y_tr, "y_valid": y_va, "y_test": y_te
    }
from copy import deepcopy
from sklearn.metrics import r2_score

def make_tabnet(cat_info, params):
    cat_idxs = [i for i, f in enumerate(features) if f in CATEGORICAL_COLUMNS]
    cat_dims = [categorical_dims[f] for f in features if f in CATEGORICAL_COLUMNS]
    cat_emb_dim = [min(params['emb'], max(4, (dim + 1)//2)) for dim in cat_dims]
    return cat_idxs, cat_dims, cat_emb_dim

def build_optimizer(optimizer_type, learning_rate, momentum, weight_decay):
    if optimizer_type == 'adam':
        optimizer_fn = torch.optim.Adam
        optimizer_params = {'lr': learning_rate, 'weight_decay': weight_decay}
    elif optimizer_type == 'adamw':
        optimizer_fn = torch.optim.AdamW
        optimizer_params = {'lr': learning_rate, 'weight_decay': weight_decay}
    elif optimizer_type == 'sgd':
        optimizer_fn = torch.optim.SGD
        optimizer_params = {'lr': learning_rate, 'momentum': momentum, 'weight_decay': weight_decay}
    elif optimizer_type == 'rmsprop':
        optimizer_fn = torch.optim.RMSprop
        optimizer_params = {'lr': learning_rate, 'momentum': momentum, 'weight_decay': weight_decay}
    return optimizer_fn, optimizer_params
def objective_regression(trial):
    # Capacidad TabNet
    n_d     = trial.suggest_int('n_d', 2, 256)
    n_a     = trial.suggest_int('n_a', 2, 256)
    n_steps = trial.suggest_int('n_steps', 1, 10)

    gamma         = trial.suggest_float('gamma', 1e-12, 1e2)
    lambda_sparse = trial.suggest_float('lambda_sparse', 1e-12, 1e2, log=True)

    batch_size  = trial.suggest_categorical('batch_size', [1024, 2048, 4096])
    mask_type   = trial.suggest_categorical('mask_type', ['entmax', 'sparsemax'])
    emb         = trial.suggest_int('emb', 3, 70)

    momentum      = trial.suggest_float('momentum', 0.001, 0.9)
    learning_rate = trial.suggest_float('learning_rate', 1e-7, 963e-1, log=True)
    weight_decay  = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)

    scheduler_gamma = trial.suggest_float('scheduler_gamma', 0.1, 0.99)
    step_size       = trial.suggest_int('step_size',  1, 20)

    virtual_batch_size = trial.suggest_categorical('virtual_batch_size', [256,512,1024])
    if isinstance(batch_size, int) and isinstance(virtual_batch_size, int) and virtual_batch_size > batch_size:
        virtual_batch_size = batch_size // 2 if batch_size >= 64 else batch_size

    optimizer_type = trial.suggest_categorical('optimizer_type', ['adam', 'adamw', 'sgd', 'rmsprop'])
    optimizer_fn, optimizer_params = build_optimizer(optimizer_type, learning_rate, momentum, weight_decay)

    p   = trial.suggest_float('p', 1e-6, 0.99)
    aug = RegressionSMOTE(p=p)

    cat_idxs = [i for i, f in enumerate(features) if f in CATEGORICAL_COLUMNS]
    cat_dims = [categorical_dims[f] for f in features if f in CATEGORICAL_COLUMNS]
    cat_emb_dim = [min(emb, max(4, (dim + 1)//2)) for dim in cat_dims]

    model = CustomTabNetRegressor(
        cat_dims=cat_dims, cat_emb_dim=cat_emb_dim, cat_idxs=cat_idxs,
        n_d=n_d, n_a=n_a, n_steps=n_steps, gamma=gamma, lambda_sparse=lambda_sparse,
        mask_type=mask_type, optimizer_fn=optimizer_fn, optimizer_params=optimizer_params,
        scheduler_params={"gamma": scheduler_gamma, "step_size": step_size},
        scheduler_fn=torch.optim.lr_scheduler.StepLR, verbose=True
    )
    model.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_name=['train', 'valid'],
        eval_metric=['mae'],
        loss_fn=my_r2_score_fn,  # (conserva tu lógica)
        max_epochs=70, patience=40,
        batch_size=batch_size, virtual_batch_size=virtual_batch_size,
        num_workers=1, drop_last=False, augmentations=aug,
    )
    mae = model.history['loss'][-1]
    return mae

def eval_and_print(title, clf_model, X_test, y_test):
    """Evalúa R² en escala original (inverse_transform) y lo imprime."""
    y_pred = clf_model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print(f"{title}: R2={r2:.4f}")
    return r2

def run_three_training_strategies(
    # modelos / kwargs
    clf_base,                   # modelo ya entrenado en la Fase 1 (con warm_start=True)
    model_init_kwargs,          # dict con los kwargs para construir un modelo nuevo idéntico (desde cero)
    # datos antiguos (Fase 1)
    X_train_old, y_train_old,   # típicamente (X_train, y_train[:,0:1]) de los 1000
    X_test_old, y_test_old,  # test y scaler usados en la Fase 1
    # datos nuevos (Fase 2)
    X_tr_new, y_tr_new,         # train de los 500
    X_va_new, y_va_new,         # valid de los 500 (para early stopping)
    X_te_new, y_te_new,  # test nuevo y su scaler
    # entrenamiento
    batch_size, virtual_batch_size, aug,
    max_epochs_ft_inc=200, patience_ft_inc=70,
    max_epochs_ft_new=200, patience_ft_new=70,
    max_epochs_scratch=200, patience_scratch=70,
    lower_lr_factor=0.1, min_lr=1e-5
):
    """
    Ejecuta:
      A) Fine-tuning incremental (old + new)
      B) Fine-tuning no incremental (solo new)
      C) Desde cero (old + new)
    y evalúa R² en test viejo y test nuevo (ambos en escala original).
    Devuelve un dict con los R².
    """
    results = {}

    # =============================
    # A) Fine-tuning incremental
    # =============================
    clf_ft_inc = deepcopy(clf_base)  # copia del clf ya entrenado
    # bajar LR para fine-tune (opcional, recomendado)
    if hasattr(clf_ft_inc, "_optimizer"):
        for g in clf_ft_inc._optimizer.param_groups:
            g["lr"] = max(g["lr"] * lower_lr_factor, min_lr)

    X_inc = np.concatenate([X_train_old, X_tr_new], axis=0)
    y_inc = np.concatenate([y_train_old, y_tr_new], axis=0)

    clf_ft_inc.fit(
        X_train=X_inc, y_train=y_inc,
        eval_set=[(X_inc, y_inc), (X_va_new, y_va_new)],
        eval_name=['train_inc', 'valid_new'],
        eval_metric=['mae'], loss_fn=my_r2_score_fn,
        max_epochs=max_epochs_ft_inc, patience=patience_ft_inc,
        batch_size=batch_size, virtual_batch_size=virtual_batch_size,
        num_workers=1, drop_last=False, augmentations=aug,
    )

    print("\n== Desempeño: Fine-tuning incremental ==")
    r2_old_inc = eval_and_print("Test viejo (FT incremental)", clf_ft_inc, X_test_old, y_test_old)
    r2_new_inc = eval_and_print("Test nuevo (FT incremental)", clf_ft_inc, X_te_new,  y_te_new)
    results["fine_tune_incremental"] = {"R2_old_test": r2_old_inc, "R2_new_test": r2_new_inc, "model": clf_ft_inc}

    # =============================
    # B) Fine-tuning no incremental (solo nuevos)
    # =============================
    clf_ft_new = deepcopy(clf_base)
    if hasattr(clf_ft_new, "_optimizer"):
        for g in clf_ft_new._optimizer.param_groups:
            g["lr"] = max(g["lr"] * lower_lr_factor, min_lr)

    clf_ft_new.fit(
        X_train=X_tr_new, y_train=y_tr_new,
        eval_set=[(X_tr_new, y_tr_new), (X_va_new, y_va_new)],
        eval_name=['train_new', 'valid_new'],
        eval_metric=['mae'], loss_fn=my_r2_score_fn,
        max_epochs=max_epochs_ft_new, patience=patience_ft_new,
        batch_size=batch_size, virtual_batch_size=virtual_batch_size,
        num_workers=1, drop_last=False, augmentations=aug,
    )

    print("\n== Desempeño: Fine-tuning NO incremental (solo nuevos) ==")
    r2_old_new = eval_and_print("Test viejo (FT no incremental)", clf_ft_new, X_test_old, y_test_old)
    r2_new_new = eval_and_print("Test nuevo (FT no incremental)", clf_ft_new, X_te_new,  y_te_new)
    results["fine_tune_only_new"] = {"R2_old_test": r2_old_new, "R2_new_test": r2_new_new, "model": clf_ft_new}

    # =============================
    # C) Desde cero (cumulative old+new)
    # =============================
    # model_init_kwargs debe contener todo lo necesario para reconstruir el TabNet
    clf_scratch = CustomTabNetRegressor(**model_init_kwargs)

    X_cum = np.concatenate([X_train_old, X_tr_new], axis=0)
    y_cum = np.concatenate([y_train_old, y_tr_new], axis=0)

    clf_scratch.fit(
        X_train=X_cum, y_train=y_cum,
        eval_set=[(X_cum, y_cum), (X_va_new, y_va_new)],
        eval_name=['train_cum', 'valid_new'],
        eval_metric=['mae'], loss_fn=my_r2_score_fn,
        max_epochs=max_epochs_scratch, patience=patience_scratch,
        batch_size=batch_size, virtual_batch_size=virtual_batch_size,
        num_workers=1, drop_last=False, augmentations=aug,
    )

    print("\n== Desempeño: Desde cero (old+new) ==")
    r2_old_sc = eval_and_print("Test viejo (desde cero)", clf_scratch, X_test_old, y_test_old)
    r2_new_sc = eval_and_print("Test nuevo (desde cero)", clf_scratch, X_te_new,  y_te_new)
    results["from_scratch"] = {"R2_old_test": r2_old_sc, "R2_new_test": r2_new_sc, "model": clf_scratch}
    return results
def pick_new_indices(n_new=500, seed=123):
    rng = np.random.RandomState(seed)
    universe = np.setdiff1d(np.arange(X.shape[0]), splits_1000["idx_sub"], assume_unique=True)
    n_new = min(n_new, universe.shape[0])
    return rng.choice(universe, size=n_new, replace=False)
Xdata = df = pd.read_pickle('/content/CHEC/SuperEventos_Criticidad_AguasAbajo_CODEs.pkl')
Xdata = Xdata[Xdata['duracion_h'] <= 100]
# ---------------------------------------------------------
# Etapa 1: seleccionar objetivo (SAIDI o SAIFI) con forma (N,1)
# Extraer variables objetivo
Dur_h = Xdata['duracion_h'].values
SAIDI = Xdata['SAIDI'].values
df1=Xdata.copy()
# Eliminar columnas no utilizadas
Xdata.drop(['inicio_evento', 'h0-solar_rad', 'h0-uv', 'h1-solar_rad', 'h1-uv', 'h2-solar_rad', 'h2-uv', 'h3-solar_rad', 'h3-uv',
            'h4-solar_rad', 'h4-uv', 'h5-solar_rad', 'h5-uv', 'h19-solar_rad', 'h19-uv', 'h20-solar_rad', 'h20-uv',
            'h21-solar_rad', 'h21-uv', 'h22-solar_rad', 'h22-uv', 'h23-solar_rad', 'h23-uv', 'evento', 'fin', 'inicio',
            'cnt_usus', 'DEP', 'MUN', 'FECHA', 'NIVEL_C', 'VALOR_C', 'TRAMOS_AGUAS_ABAJO', 'EQUIPOS_PUNTOS',
            'PUNTOS_POLIGONO', 'LONGITUD2', 'LATITUD2', 'FECHA_C','TRAMOS_AGUAS_ABAJO_CODES','ORDER_'],
           inplace=True, axis=1)

# Definir la variable objetivo y eliminarla del conjunto de características
target = ['SAIFI', 'SAIDI', 'duracion_h']
y1 = Xdata[target].values
Xdata.drop(target, axis=1, inplace=True)
y = y1[:, 0:1].astype('float32')

# Copia de trabajo de X
df = Xdata.copy()

# ---------------------------------------------------------
# Etapa 2: tipificar columnas
NUMERIC_COLUMNS = df.select_dtypes(include=['number']).columns.tolist()
CATEGORICAL_COLUMNS = df.select_dtypes(include=['object', 'category']).columns.tolist()

# ---------------------------------------------------------
# Etapa 3: imputación numérica
max_values = {}
for col in NUMERIC_COLUMNS:
    max_value = pd.to_numeric(df[col], errors='coerce').max()
    if pd.isna(max_value):
        max_value = 0.0
    max_values[col] = max_value
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(-10.0 * max_value)

# ---------------------------------------------------------
# Etapa 4: codificación categórica
label_encoders = {}
categorical_dims = {}
for col in CATEGORICAL_COLUMNS:
    enc = LabelEncoder()
    s = df[col].astype(str).fillna("no aplica")
    enc.fit(s)
    df[col] = enc.transform(s)
    label_encoders[col] = enc
    categorical_dims[col] = len(enc.classes_)

# ---------------------------------------------------------
# Etapa 5: construir matrices X, y
unused_feat = []
# Si Xdata NO incluye el target, basta con tomar todas las columnas
features = [c for c in df.columns if c not in unused_feat]
X = df[features].values.astype('float32')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1o_fZIhk6ErrtrM3eVZPF9s2qj8l4FoqS
From (redirected): https://drive.google.com/uc?id=1o_fZIhk6ErrtrM3eVZPF9s2qj8l4FoqS&confirm=t&uuid=a88678a6-38f4-475b-b1aa-874aa7424642
To: /content/SuperEventos_Criticidad_AguasAbajo_CODEs.zip
100% 214M/214M [00:01<00:00, 123MB/s]
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pas

/usr/local/lib/python3.12/dist-packages/cudf/utils/gpu_utils.py:75: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))


# Incorporación de nuevas características

In [ ]:
Xdata = df = pd.read_pickle('/content/CHEC/SuperEventos_Criticidad_AguasAbajo_CODEs.pkl')
Xdata = Xdata[Xdata['duracion_h'] <= 100]
Rayos=pd.read_pickle('/content/CHEC/Data_CHEC/Rayos.pkl')
Rayos['FECHA'] = pd.to_datetime(Rayos['FECHA'])
Vegetacion=pd.read_pickle('/content/CHEC/Data_CHEC/Vegetacion.pkl')
Vegetacion['LATITUD'] = Vegetacion['LATITUD'].astype(float)
Vegetacion['LONGITUD'] = Vegetacion['LONGITUD'].astype(float)

In [ ]:
Rayos.head()

,ID,LATITUD,LONGITUD,ALTITUD,TIPO,CORRIENTE,ERROR,CODE,FPARENT,DISTANCIA_A_NODO,FECHA,DEP,MUN
0,202009191522446,4.9487,-75.5995,"12,4",2,-5.3,"0,016",1657,CHA23L16,78.252413,2020-09-03 14:15:22,CALDAS,VILLAMARÍA
1,2020076142486,4.9488,-75.7033,0,1,-5.8,"0,061",58412,INS23L13,24.710599,2020-07-10 01:01:42,RISARALDA,SANTA ROSA DE CABAL
2,20200784256395,4.9488,-75.6958,"6,6",2,5.2,"0,016",4244,INS23L13,35.126961,2020-07-04 03:42:56,RISARALDA,SANTA ROSA DE CABAL
3,202008193652495,4.6288,-75.6113,"10,1",2,5.0,"0,003",A186,ROS40L21,289.085969,2020-08-22 14:36:52,QUINDÍO,CIRCASIA
4,20200823350497,4.9488,-76.0051,0,1,-24.7,"0,145",36021,BOA23L14,354.305040,2020-08-13 18:35:04,RISARALDA,BALBOA


In [ ]:
Rayos.columns

Index(['ID', 'LATITUD', 'LONGITUD', 'ALTITUD', 'TIPO', 'CORRIENTE', 'ERROR',
       'CODE', 'FPARENT', 'DISTANCIA_A_NODO', 'FECHA', 'DEP', 'MUN'],
      dtype='object')

In [ ]:
Vegetacion.head()

,NOM_COMUN,TIPO_VEGET,ESTADO_INICIAL,FECHA,LADO_RED,DAP_ESTIM,LONG_INTER,TIPO_INTER,NIVEL_RIES,CIRCUITO_TRAMO,NODO_1,NODO_2,LONGITUD,LATITUD,DEP,MUN
0,YARUMO,Bosque natural,Nuevo,2023-11-30,Debajo,22,6,Rocería,Medio,ESM40L27,A06084,A06085,-75.714305,5.013050,RISARALDA,MARSELLA
1,GUADUA,Guadual,Nuevo,2023-11-30,Debajo,11,6,Tala,Medio,ESM40L27,A06089,A06090,-75.726387,5.023843,RISARALDA,MARSELLA
2,MANDARINO,Frutales,Nuevo,2023-11-30,Debajo,28,6,Poda,Alto,ESM40L28,A06059,A06060,-75.645768,4.992025,CALDAS,CHINCHINÁ
3,GUAMO,Cafe/sombrio,Nuevo,2023-11-30,Debajo,45,6,Poda,Medio,ESM40L27,A06070,A06071,-75.674651,4.993295,CALDAS,CHINCHINÁ
4,GUADUA,Guadual,Nuevo,2023-11-30,Debajo,12,6,Tala,Medio,ESM40L27,A06087,A06088,-75.723988,5.018694,RISARALDA,MARSELLA


In [ ]:
Vegetacion.columns

Index(['NOM_COMUN', 'TIPO_VEGET', 'ESTADO_INICIAL', 'FECHA', 'LADO_RED',
       'DAP_ESTIM', 'LONG_INTER', 'TIPO_INTER', 'NIVEL_RIES', 'CIRCUITO_TRAMO',
       'NODO_1', 'NODO_2', 'LONGITUD', 'LATITUD', 'DEP', 'MUN'],
      dtype='object')

In [ ]:
Xdata.shape

(166323, 355)

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree
from pandas.api.types import is_numeric_dtype
from ast import literal_eval
from tqdm import tqdm

def enriquecer_eventos_con_rayos_y_vegetacion(
    Eventos: pd.DataFrame,
    Rayos: pd.DataFrame,
    Vegetacion: pd.DataFrame,
    *,
    radio_rayos: float = 0.005,        # ~0.5 km en grados aprox.
    ventana_dias: int = 1,             # [inicio - ventana_dias, inicio]
    radio_vegetacion: float = 0.0003,  # ~30 m aprox. en grados
    veg_vars: list | None = None,      # variables de interés en Vegetación
    usar_tqdm: bool = True,
    col_lat: str = 'LATITUD',
    col_lon: str = 'LONGITUD'
) -> pd.DataFrame:
    """
    Para cada evento, toma TODAS las (LATITUD,LONGITUD) únicas de su municipio (MUN)
    como puntos de consulta y busca alrededor:
      - RAYOS: dentro de radio_rayos y en la ventana temporal [inicio - ventana_dias, inicio].
      - VEGETACIÓN: dentro de radio_vegetacion (solo espacial).

    Vegetación:
      * 'conteo_vegetacion'
      * Para cada var en veg_vars:
          - Si es numérica (o numérico-like): {mean, median, min, max, std} (std=0 si 1 dato)
          - Si es categórica: {mode}. Si var == 'NOM_COMUN' -> columna 'nombre_comun_mas_frecuente'
    """

    # ---------------------------
    # Normalización de insumos
    # ---------------------------
    Eventos = Eventos.copy()

    # Asegurar tipos numéricos de lat/lon y fecha en Rayos / Vegetación
    for df in (Rayos, Vegetacion, Eventos):
        for c in (col_lat, col_lon):
            if c in df.columns:
                df[c] = pd.to_numeric(df[c], errors='coerce')

    if 'FECHA' in Rayos.columns:
        Rayos['FECHA'] = pd.to_datetime(Rayos['FECHA'], errors='coerce')
    if 'inicio' in Eventos.columns:
        Eventos['inicio'] = pd.to_datetime(Eventos['inicio'], errors='coerce')

    if 'MUN' not in Eventos.columns:
        raise ValueError("Eventos debe contener la columna 'MUN'.")

    # ---------------------------
    # Helpers
    # ---------------------------
    def _build_kd_by_group(df, key_col='MUN', lat=col_lat, lon=col_lon):
        """Construye KDTree por grupo, alineado a índices tras dropna."""
        trees = {}
        for k, g in df.groupby(key_col):
            mask = g[[lat, lon]].notna().all(axis=1)
            g_f = g.loc[mask]
            if not g_f.empty:
                coords = g_f[[lat, lon]].to_numpy()
                trees[k] = (cKDTree(coords), g_f)
        return trees

    def _query_indices_for_points(tree, points, r):
        """Une índices cercanos para múltiples puntos."""
        idxs = set()
        for (lat, lon) in points:
            try:
                lat = float(lat); lon = float(lon)
            except Exception:
                continue
            idxs.update(tree.query_ball_point([lat, lon], r=r))
        return idxs

    def _std_safe(s: pd.Series):
        s = pd.to_numeric(s, errors='coerce').dropna()
        return 0.0 if len(s) <= 1 else float(s.std())

    def _is_numeric_like(series: pd.Series) -> bool:
        """Decide si tratar una variable como numérica (dtype numérico o ≥60% convertible)."""
        if is_numeric_dtype(series):
            return True
        s_num = pd.to_numeric(series, errors='coerce')
        return s_num.notna().mean() >= 0.60

    def _veg_out_cols_for(var: str, kind: str) -> list[str]:
        """Define columnas de salida para cada variable."""
        if kind == 'numeric':
            return [f'{var}_mean', f'{var}_median', f'{var}_min', f'{var}_max', f'{var}_std']
        # categórica
        if var == 'NOM_COMUN':
            return ['nombre_comun_mas_frecuente']
        return [f'{var}_mode']

    def _veg_empty_vals_for(kind: str) -> list:
        if kind == 'numeric':
            return [np.nan, np.nan, np.nan, np.nan, 0.0]
        return [np.nan]  # categórica

    # ---------------------------
    # Puntos por municipio (cache)
    # ---------------------------
    pts_cache_mun: dict = {}
    if (col_lat in Eventos.columns) and (col_lon in Eventos.columns):
        tmp = (
            Eventos[['MUN', col_lat, col_lon]]
            .dropna()
            .drop_duplicates()
        )
        for k, g in tmp.groupby('MUN'):
            pts_cache_mun[k] = [ (float(a), float(b)) for a, b in g[[col_lat, col_lon]].to_numpy() ]
    else:
        # Si no hay LATITUD/LONGITUD en Eventos, no hay puntos municipio-latlon
        pts_cache_mun = {}

    # ---------------------------------------------------------------------
    # (1) RAYOS
    # ---------------------------------------------------------------------
    rayos_trees = _build_kd_by_group(Rayos, key_col='MUN', lat=col_lat, lon=col_lon)

    cols_rayos = [
        'ALTITUD_mean', 'ALTITUD_median', 'ALTITUD_min', 'ALTITUD_max', 'ALTITUD_std',
        'CORRIENTE_mean', 'CORRIENTE_median', 'CORRIENTE_min', 'CORRIENTE_max', 'CORRIENTE_std',
        'TIPO_1_count', 'TIPO_2_count'
    ]
    out_rayos = []

    iterator = Eventos.itertuples()
    pbar = tqdm(total=len(Eventos), desc='Rayos (por lat/lon de municipio)') if usar_tqdm else None
    for ev in iterator:
        mun = getattr(ev, 'MUN', None)
        inicio = pd.to_datetime(getattr(ev, 'inicio', pd.NaT), errors='coerce')
        puntos = pts_cache_mun.get(mun, [])

        if not mun or mun not in rayos_trees or not puntos or pd.isna(inicio):
            out_rayos.append([np.nan]*len(cols_rayos))
            if pbar: pbar.update(1)
            continue

        _, rayos_mun = rayos_trees[mun]
        rayos_mun = rayos_mun.dropna(subset=[col_lat, col_lon, 'FECHA'])
        if rayos_mun.empty:
            out_rayos.append([np.nan]*len(cols_rayos))
            if pbar: pbar.update(1)
            continue

        rayos_temp = rayos_mun[(rayos_mun['FECHA'] >= inicio - pd.Timedelta(days=ventana_dias)) &
                               (rayos_mun['FECHA'] <= inicio)]
        if rayos_temp.empty:
            out_rayos.append([np.nan]*len(cols_rayos))
            if pbar: pbar.update(1)
            continue

        tree_temp = cKDTree(rayos_temp[[col_lat, col_lon]].to_numpy())
        idxs = _query_indices_for_points(tree_temp, puntos, r=radio_rayos)
        if not idxs:
            out_rayos.append([np.nan]*len(cols_rayos))
            if pbar: pbar.update(1)
            continue

        sub = rayos_temp.iloc[list(idxs)].copy()
        if 'ALTITUD' in sub.columns:
            sub['ALTITUD'] = pd.to_numeric(sub['ALTITUD'], errors='coerce')
        else:
            sub['ALTITUD'] = np.nan

        if 'CORRIENTE' in sub.columns:
            sub['CORRIENTE'] = pd.to_numeric(sub['CORRIENTE'], errors='coerce').abs()
        else:
            sub['CORRIENTE'] = np.nan

        tipo_col = 'TIPO' if 'TIPO' in sub.columns else None
        out_rayos.append([
            sub['ALTITUD'].mean(), sub['ALTITUD'].median(), sub['ALTITUD'].min(), sub['ALTITUD'].max(), _std_safe(sub['ALTITUD']),
            sub['CORRIENTE'].mean(), sub['CORRIENTE'].median(), sub['CORRIENTE'].min(), sub['CORRIENTE'].max(), _std_safe(sub['CORRIENTE']),
            (sub[tipo_col] == 1).sum() if tipo_col else np.nan,
            (sub[tipo_col] == 2).sum() if tipo_col else np.nan,
        ])

        if pbar: pbar.update(1)
    if pbar: pbar.close()

    df_rayos = pd.DataFrame(out_rayos, columns=cols_rayos, index=Eventos.index)
    Eventos.loc[df_rayos.index, df_rayos.columns] = df_rayos

    # ---------------------------------------------------------------------
    # (2) VEGETACIÓN (dinámico por veg_vars)
    # ---------------------------------------------------------------------
    if veg_vars is None:
        veg_vars = ['NOM_COMUN'] if 'NOM_COMUN' in Vegetacion.columns else []

    # Clasificar variables (forzar 'NOM_COMUN' como categórica)
    veg_specs = []
    for var in veg_vars:
        if var not in Vegetacion.columns:
            veg_specs.append((var, 'missing'))
        else:
            if var == 'NOM_COMUN':
                kind = 'cat'
            else:
                kind = 'numeric' if _is_numeric_like(Vegetacion[var]) else 'cat'
            veg_specs.append((var, kind))

    # Armar columnas de salida
    cols_veg = ['conteo_vegetacion']
    for var, kind in veg_specs:
        cols_veg += _veg_out_cols_for(var, kind if kind != 'missing' else 'cat')

    veg_trees = _build_kd_by_group(Vegetacion, key_col='MUN', lat=col_lat, lon=col_lon)
    out_veg = []

    def _compute_veg_row(sub_df: pd.DataFrame) -> list:
        row_vals = [len(sub_df)]  # conteo_vegetacion
        for var, kind in veg_specs:
            if var not in sub_df.columns or sub_df.empty:
                row_vals += _veg_empty_vals_for(kind if kind != 'missing' else 'cat')
                continue

            if kind == 'numeric':
                s = pd.to_numeric(sub_df[var], errors='coerce')
                row_vals += [
                    s.mean(), s.median(), s.min(), s.max(),
                    (0.0 if s.dropna().shape[0] <= 1 else float(s.std()))
                ]
            else:  # categórica (incluye NOM_COMUN)
                s = sub_df[var].dropna()
                moda = s.mode()
                row_vals += [ (moda.iloc[0] if not moda.empty else np.nan) ]
        return row_vals

    iterator = Eventos.itertuples()
    pbar = tqdm(total=len(Eventos), desc='Vegetación (por lat/lon de municipio)') if usar_tqdm else None
    for ev in iterator:
        mun = getattr(ev, 'MUN', None)
        puntos = pts_cache_mun.get(mun, [])

        if not mun or mun not in veg_trees or not puntos:
            empty_vals = [0]
            for _, kind in veg_specs:
                empty_vals += _veg_empty_vals_for(kind if kind != 'missing' else 'cat')
            out_veg.append(empty_vals)
            if pbar: pbar.update(1)
            continue

        _, veg_mun = veg_trees[mun]
        veg_mun = veg_mun.dropna(subset=[col_lat, col_lon])
        if veg_mun.empty:
            empty_vals = [0]
            for _, kind in veg_specs:
                empty_vals += _veg_empty_vals_for(kind if kind != 'missing' else 'cat')
            out_veg.append(empty_vals)
            if pbar: pbar.update(1)
            continue

        tree_veg = cKDTree(veg_mun[[col_lat, col_lon]].to_numpy())
        idxs = _query_indices_for_points(tree_veg, puntos, r=radio_vegetacion)
        if not idxs:
            empty_vals = [0]
            for _, kind in veg_specs:
                empty_vals += _veg_empty_vals_for(kind if kind != 'missing' else 'cat')
            out_veg.append(empty_vals)
            if pbar: pbar.update(1)
            continue

        sub = veg_mun.iloc[list(idxs)]
        out_veg.append(_compute_veg_row(sub))
        if pbar: pbar.update(1)
    if pbar: pbar.close()

    df_veg = pd.DataFrame(out_veg, columns=cols_veg, index=Eventos.index)
    Eventos.loc[df_veg.index, df_veg.columns] = df_veg

    return Eventos


In [ ]:
Xdata=enriquecer_eventos_con_rayos_y_vegetacion(Xdata, Rayos, Vegetacion,ventana_dias= 24,veg_vars=['NOM_COMUN','ESTADO_INICIAL','LADO_RED','DAP_ESTIM','LONG_INTER','TIPO_INTER', 'NIVEL_RIES'])
Xdata.to_pickle('/content/drive/MyDrive/Colab Notebooks/Diego/SuperEventos_Criticidad_AguasAbajo_CODEs.pkl')
Xdata.shape

Vegetación (por lat/lon de municipio): 100%|██████████| 166323/166323 [30:24<00:00, 91.17it/s] 


(166323, 369)